### Part 1.5 - Extract_data_25000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Fifth 5000 movies (25,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the fifth 5000 movies and create the souplist of those URLs
"""
content_five = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_five_13012020.pkl'), 'rb') as f:
    
    content_five = pickle.load(f)

print("Number of URLs: {}".format(len(content_five)))

content_souplist_five = []

for i in tqdm_notebook(content_five):
    
    content_souplist_five.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_five)))

In [ ]:
"""
Open the synopsis urls of the fifth 5000 movies and create the souplist of those URLs
"""
synopsis_five = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_five_13012020.pkl'), 'rb') as f:
    
    synopsis_five = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_five)))

synopsis_souplist_five = []

for i in tqdm_notebook(synopsis_five):
    
    synopsis_souplist_five.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_five)))

In [ ]:
"""
Open the reviews urls of the fifth 5000 movies and create the souplist of those URLs
"""
review_five = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_five_15012020.pkl'), 'rb') as f:
    
    review_five = pickle.load(f)

print("Number of URLs: {}".format(len(review_five)))

review_souplist_five = []

for i in tqdm_notebook(review_five):
    
    review_souplist_five.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_five)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_five)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_five)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_five)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_five)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_five)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_five)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_five_13012020.pkl'), 'rb') as f:
    
    content_five = pickle.load(f)

print("Number of URLs: {}".format(len(content_five)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_five_15012020.pkl'), 'rb') as f:
    
    review_five = pickle.load(f)

print("Number of URLs: {}".format(len(review_five)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\plot_five_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\rating_five_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\actors_five_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\director_five_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\synopsis_five_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\pre-indexed files\\reviews_five_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[183,366,374,465,484,522,567,571,580,594,606,623,630,707,711,729,733,735,747,750,753,780,864,875,1041,1043,1050,1093,1249,1250,1274,1373,1427,1457,1486,1503,1508,1553,1610,1662,1679,1693,1748,1820,1871,1893,1990,2036,2133,2158,2166,2231,2250,2395,2510,2886,2963,2972,3071,3189,3243,3283,3295,3298,3329,3396,3410,3429,3451,3485,3555,3672,3754,3819,3834,3934,3936,3973,4068,4077,4216,4227,4551,4564,4614,4645,4646,4674,4712,4720,4746,4787,4852,4858,4882,4889,4942,4957,4978,4995,112,115,183,366,374,501,747,1487,1674,2400,4089,4843,12,70,89,90,91,196,214,215,242,248,319,330,366,381,403,410,432,505,518,550,581,624,684,744,746,789,823,824,829,908,921,968,1097,1166,1265,1288,1325,1363,1381,1422,1527,1535,1575,1622,1674,1936,1989,2026,2067,2122,2220,2299,2340,2396,2400,2434,2435,2517,2685,2686,2693,2704,2724,2731,2748,2770,2907,2949,2953,3117,3126,3189,3280,3293,3337,3398,3435,3533,3555,3584,3645,3709,3756,3844,3845,3863,4016,4122,4172,4213,4225,4369,4450,4457,4476,4489,4498,4539,4551,4578,4646,4689,4691,4732,4843,4855,4860,4927,4942,4954,4962,4978]

index_remove = list(dict.fromkeys(index_remove))

print(len(index_remove))

content_index_five = [i for j, i in enumerate(content_five) if j not in index_remove]

review_index_five = [i for j, i in enumerate(review_five) if j not in index_remove]

print(len(content_index_five))

print(len(review_index_five))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_five\\content_index_five_20012020.pkl'), 'wb') as f: 
#     pickle.dump(content_index_five, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_five\\review_index_five_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_five, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_five = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\content_index_five_20012020.pkl'), 'rb') as f:
    
    content_index_five = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_five)))

for i in tqdm_notebook(content_index_five):
    
    content_souplist_five.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_five)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\review_index_five_20012020.pkl'), 'rb') as f:
    
    review_five = pickle.load(f)

print("Number of URLs: {}".format(len(review_five)))

review_souplist_five = []

for i in tqdm_notebook(review_five):
    
    review_souplist_five.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_five)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_five)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_five)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_five)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_five)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating)

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_five)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\plot_five_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\actors_five_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\director_five_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\reviews_five_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_five, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\rating_five_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[183,366,374,465,484,522,567,571,580,594,606,623,630,707,711,729,733,735,747,750,753,780,864,875,1041,1043,1050,1093,1249,1250,1274,1373,1427,1457,1486,1503,1508,1553,1610,1662,1679,1693,1748,1820,1871,1893,1990,2036,2133,2158,2166,2231,2250,2395,2510,2886,2963,2972,3071,3189,3243,3283,3295,3298,3329,3396,3410,3429,3451,3485,3555,3672,3754,3819,3834,3934,3936,3973,4068,4077,4216,4227,4551,4564,4614,4645,4646,4674,4712,4720,4746,4787,4852,4858,4882,4889,4942,4957,4978,4995,112,115,183,366,374,501,747,1487,1674,2400,4089,4843,12,70,89,90,91,196,214,215,242,248,319,330,366,381,403,410,432,505,518,550,581,624,684,744,746,789,823,824,829,908,921,968,1097,1166,1265,1288,1325,1363,1381,1422,1527,1535,1575,1622,1674,1936,1989,2026,2067,2122,2220,2299,2340,2396,2400,2434,2435,2517,2685,2686,2693,2704,2724,2731,2748,2770,2907,2949,2953,3117,3126,3189,3280,3293,3337,3398,3435,3533,3555,3584,3645,3709,3756,3844,3845,3863,4016,4122,4172,4213,4225,4369,4450,4457,4476,4489,4498,4539,4551,4578,4646,4689,4691,4732,4843,4855,4860,4927,4942,4954,4962,4978]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_five = dataset.iloc[20000:25000].reset_index(drop=True)
dataset_five = dataset_five[~dataset_five.index.isin(index_remove)]
dataset_five.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the fifth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\plot_five_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\rating_five_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\actors_five_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\director_five_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_five\\reviews_five_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_five['actors'] = actors
dataset_five['plot'] = plot
dataset_five['imdb_rating'] = rating
dataset_five['director'] = director
dataset_five['reviews'] = reviews
dataset_five = dataset_five.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[47,52,57,110,111,149,204,305,344,361,375,438,491,542,564,568,570,573,595,597,610,630,631,635,682,689,711,712,715,732,734,793,861,893,897,971,974,975,1070,1082,1110,1135,1137,1156,1170,1199,1204,1209,1284,1294,1301,1307,1386,1404,1453,1471,1483,1516,1523,1551,1576,1652,1656,1676,1699,1700,1724,1769,1778,1787,1816,1818,1835,1852,1869,1887,1910,1942,1970,1988,1996,2003,2018,2032,2068,2078,2090,2094,2208,2220,2242,2244,2263,2264,2312,2353,2356,2360,2385,2400,2445,2462,2463,2535,2570,2572,2573,2576,2577,2584,2603,2620,2628,2633,2683,2694,2727,2764,2820,2868,2876,2904,2946,2966,2977,2984,3003,3006,3023,3067,3075,3081,3092,3105,3147,3211,3280,3295,3296,3326,3358,3359,3363,3365,3367,3374,3394,3405,3432,3433,3447,3523,3524,3526,3564,3589,3731,3790,3815,3842,3845,3851,3902,3903,3928,3935,3958,3976,3980,3987,4038,4108,4150,4152,4189,4222,4244,4263,4270,4273,4295,4297,4304,4323,4324,4358,4393,4435,4490,4493,4496,4501,4535,4555,4576,4656,4669,4724,4738,4747,4761,4778]
dataset_five = dataset_five[~dataset_five.index.isin(index_remove)]
dataset_five = dataset_five[dataset_five.astype(str)['reviews'] != '[]']
dataset_five.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_five.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_five\\dataset_five_final_25012020.pkl"))